# 🚀 Day 4-2: 스마트한 모델 튜닝의 시작, Optuna HPO 🧠

최고의 성능을 내는 머신러닝 모델을 만드는 과정은 종종 최고의 요리 레시피를 찾는 여정과 비유됩니다. 

좋은 재료(데이터)를 준비하고, 기본 조리법(알고리즘)을 선택했다면, 다음은 양념의 '황금 비율'을 찾는 단계입니다. 

소금을 얼마나 넣을지, 불의 세기는 어떻게 조절할지 등 사소해 보이는 설정 하나하나가 요리의 맛을 좌우하듯, 

머신러닝에서도 이러한 '설정값'들이 모델의 성능을 결정합니다.

이 설정값들을 우리는 `하이퍼파라미터(Hyperparameter)` 라고 부릅니다.  

예를 들어, 랜덤 포레스트의 트리 개수(`n_estimators`)나 트리의 최대 깊이(`max_depth`), 신경망의 학습률(`learning_rate`) 등이 모두 하이퍼파라미터에 해당합니다.  

이 값들은 모델이 학습을 시작하기 전에 개발자가 직접 정해주어야 하며, 어떤 값을 선택하느냐에 따라 모델의 성능이 극적으로 달라질 수 있습니다. 

그렇다면 최적의 하이퍼파라미터 조합은 어떻게 찾을 수 있을까요? 

모든 경우의 수를 하나씩 시도해보는 것은 하이퍼파라미터가 몇 개만 되어도 조합의 수가 기하급수적으로 늘어나기 때문에 현실적으로 불가능에 가깝습니다. 

이번 시간에는 이 문제를 해결하기 위한 세련되고 강력한 도구, `Optuna(옵튜나)`에 대해 배우고, 이를 활용해 하이퍼파라미터 최적화(Hyperparameter Optimization, HPO)를 자동화하는 방법을 익혀보겠습니다.


---

### 1. 하이퍼파라미터 탐색: 무작위에서 베이지안까지

최적의 하이퍼파라미터를 찾기 위한 여정에는 여러 전략이 있습니다. 가장 기본적인 두 가지 방법과 그 한계를 먼저 짚어보겠습니다.

#### 🧠 개념 이해하기: Grid Search vs. Random Search

* `그리드 탐색 (Grid Search)`: 개발자가 지정한 하이퍼파라미터 값들의 목록을 받아, 가능한 모든 조합을 격자(Grid)처럼 만들어 하나도 빠짐없이 테스트하는 가장 단순하고 무식한(?) 방법입니다.  모든 조합을 시도하므로 최고의 조합을 놓칠 염려는 없지만, 탐색해야 할 조합이 조금만 늘어나도 시간이 기하급수적으로 증가하여 매우 비효율적입니다. 

* `랜덤 탐색 (Random Search)`: 지정된 범위 내에서 하이퍼파라미터 조합을 무작위로 샘플링하여 테스트하는 방법입니다.  그리드 탐색보다 중요하지 않은 파라미터에 낭비되는 시간을 줄이고, 의외의 좋은 조합을 발견할 가능성이 있어 동일한 시간 내에 더 나은 성능을 보이는 경우가 많습니다.  하지만 이 방법 역시 이전 시도 결과로부터 아무것도 배우지 못하고 그저 '운'에 맡기는 방식이라는 한계가 있습니다. 

이러한 한계를 극복하기 위해 등장한 것이 바로 `베이지안 최적화(Bayesian Optimization)` 입니다.

#### 🧠 개념 이해하기: 베이지안 최적화

`베이지안 최적화`는 '과거의 경험으로부터 학습하여 더 나은 의사결정을 내리는' 스마트한 탐색 기법입니다.  

이전 하이퍼파라미터 조합들의 시도 결과를 바탕으로, 다음번에 시도할 때 가장 성능이 좋을 것으로 '기대되는' 유망한 지점을 통계적으로 추론하여 탐색을 진행합니다. 

이 방식은 두 가지 핵심 요소의 균형을 맞추며 동작합니다.

1.  `탐색 (Exploration)`: 아직 시도해보지 않은 불확실한 영역을 탐험하여 의외의 '대박'을 노립니다.
   
2.  `활용 (Exploitation)`: 지금까지 가장 좋았던 결과 주변을 더 깊게 파고들어 점진적인 성능 향상을 꾀합니다.

이처럼 과거 데이터를 기반으로 다음 탐색 지점을 지능적으로 선택하기 때문에, 그리드 탐색이나 랜덤 탐색에 비해 훨씬 적은 시도만으로도 전역 최적(Global Optimum)에 가까운 하이퍼파라미터 조합을 찾아낼 수 있습니다. 

그리고 `Optuna`는 바로 이 베이지안 최적화를 기반으로 한, 사용하기 매우 편리한 파이썬 라이브러리입니다. 


#### 💻 Optuna 설치하기

Optuna는 `pip`을 통해 간단히 설치할 수 있습니다. 시각화 기능까지 함께 사용하기 위해 `plotly`도 같이 설치해 줍니다. 

In [ ]:
# 터미널이나 Jupyter Notebook 셀에서 실행하세요.
!pip install optuna plotly

---

### 2. Optuna의 기본 구조: `Study`, `Objective`, `Trial`

Optuna를 사용한 최적화는 크게 세 가지 요소를 중심으로 이루어집니다. 

1.  `하이퍼파라미터(Hyperparameter)`: 최적화의 '목표'를 정의하는 함수입니다. 이 함수는 특정 하이퍼파라미터 조합을 입력받아 모델의 성능(예: 정확도, RMSE 등)을 계산하고 반환하는 역할을 합니다.  Optuna는 이 함수의 반환값을 최대화하거나 최소화하는 방향으로 탐색을 진행합니다.

2.  `Optuna(옵튜나)`: `Objective` 함수 내에서 하이퍼파라미터 값을 제안하는 역할을 합니다.  예를 들어, `trial.suggest_int('n_estimators', 100, 500)`와 같은 코드는 'n_estimators'라는 이름의 파라미터를 100에서 500 사이의 정수 중에서 추천해달라는 의미입니다.

3.  `그리드 탐색 (Grid Search)`: 전체 최적화 과정을 관리하고 조율하는 컨트롤 타워입니다. 어떤 방향(최대화/최소화)으로 최적화를 진행할지 설정하고, 각 `trial`의 기록을 모두 저장하며, 최종적으로 최적의 결과를 알려줍니다. 

#### 💻 코드 예시: 간단한 수학 함수 최적화

개념을 확실히 이해하기 위해, 머신러닝 모델이 아닌 간단한 2차 함수 $f(x) = (x-2)^2$의 최솟값을 찾는 문제에 Optuna를 적용해 보겠습니다. 이 함수의 최솟값은 $x=2$일 때 $0$이 됩니다. 

In [1]:
import optuna

# 1. Objective 함수 정의
def objective_math(trial):
    # Trial 객체가 -10과 10 사이에서 float 타입의 x값을 제안
    x = trial.suggest_float('x', -10, 10)

    # 우리가 최소화하고 싶은 목표 함수
    y = (x - 2)**2

    # 성능 지표(y값)를 반환
    return y

# 2. Study 객체 생성
# direction='minimize'로 설정하여 objective 함수의 반환값을 최소화하는 것을 목표로 함
study = optuna.create_study(direction='minimize')

# 3. 최적화 실행
# objective 함수를 100번 시도(trial)하여 최적의 x값을 찾음
study.optimize(objective_math, n_trials=100)

# 최적화 결과 확인
print("최적의 x 값:", study.best_params)
print("해당 x에서의 함수 최솟값:", study.best_value)

[I 2025-06-20 08:09:25,829] A new study created in memory with name: no-name-7a33f3e2-6637-4b4e-841a-bf05cb8882fd
[I 2025-06-20 08:09:25,834] Trial 0 finished with value: 20.104380141138716 and parameters: {'x': -2.4837908226342043}. Best is trial 0 with value: 20.104380141138716.
[I 2025-06-20 08:09:25,836] Trial 1 finished with value: 26.39958785234146 and parameters: {'x': -3.1380529242448896}. Best is trial 0 with value: 20.104380141138716.
[I 2025-06-20 08:09:25,836] Trial 2 finished with value: 140.19971209348276 and parameters: {'x': -9.8405959348963}. Best is trial 0 with value: 20.104380141138716.
[I 2025-06-20 08:09:25,840] Trial 3 finished with value: 8.90617061238132 and parameters: {'x': -0.9843207958229492}. Best is trial 3 with value: 8.90617061238132.
[I 2025-06-20 08:09:25,844] Trial 4 finished with value: 22.271451685634354 and parameters: {'x': -2.719263892349564}. Best is trial 3 with value: 8.90617061238132.
[I 2025-06-20 08:09:25,845] Trial 5 finished with value: 

최적의 x 값: {'x': 2.0052552472940235}
해당 x에서의 함수 최솟값: 2.761762412134122e-05


위 코드를 실행하면 Optuna가 100번의 시도를 통해 함수 값이 0에 가장 가까워지는 $x$값, 즉 2.0에 매우 근접한 값을 찾아내는 것을 확인할 수 있습니다. 

#### ✏️ 연습문제 1

아래에 정의된 3차 함수 $f(x) = x^3 - 6x^2 + 9x + 1$ 의 최솟값을 $-1$과 $4$ 사이의 범위에서 찾아보세요. Optuna를 사용하여 `objective` 함수를 정의하고, 50번의 `trial`을 통해 최솟값과 그때의 $x$값을 출력하는 코드를 작성하세요.

In [ ]:
# 여기에 코드를 작성하세요.
def objective_poly(trial):
    # -1과 4 사이에서 x값을 제안받으세요.
    # 3차 함수식을 계산하여 반환하세요.
    return # 여기에 식 작성

# study 객체를 생성하고 'minimize' 방향으로 설정하세요.
study_poly = ??

# 50번의 trial로 최적화를 실행하세요.

# 결과를 출력하세요.
print("Best params:", study_poly.best_params)
print("Best value:", study_poly.best_value)

---

### 3. 실전! 머신러닝 모델에 Optuna 적용하기

이제 실제 머신러닝 모델의 하이퍼파라미터 튜닝에 Optuna를 적용해 보겠습니다. 

`scikit-learn`의 와인 데이터셋(Wine dataset)을 사용하여 `RandomForestClassifier`의 분류 정확도를 극대화하는 최적의 하이퍼파라미터 조합을 찾아보겠습니다.

#### 🧠 개념 이해하기: 탐색 공간(Search Space) 정의

Optuna의 `trial` 객체는 다양한 타입의 하이퍼파라미터 값을 제안하는 여러 메소드를 제공합니다.

* `trial.suggest_int(name, low, high)`: `low`와 `high` 사이의 `하이퍼파라미터(Hyperparameter)`를 추천합니다. (예: 트리의 개수)
  
* `trial.suggest_float(name, low, high)`: `low`와 `high` 사이의 `Optuna(옵튜나)`를 추천합니다. (예: Regularization 강도)
* `trial.suggest_categorical(name, choices)`: 주어진 리스트(`choices`) 안에서 하나의 `그리드 탐색 (Grid Search)`를 선택합니다. (예: `['gini', 'entropy']`)
* `trial.suggest_loguniform(name, low, high)`: 로그 스케일에서 균등하게 `랜덤 탐색 (Random Search)`를 추천합니다. $10^{-5}$ 에서 $10^{-1}$ 처럼 매우 넓은 범위의 학습률(`learning_rate`) 등을 탐색할 때 유용합니다.

#### 💻 코드 예시: RandomForestClassifier 튜닝

`RandomForestClassifier`의 주요 하이퍼파라미터인 `n_estimators`, `max_depth`, `min_samples_split` 등을 Optuna로 최적화해 보겠습니다. 

모델의 성능은 3-Fold 교차 검증(Cross-validation)을 통한 평균 정확도로 평가합니다. 

In [2]:
import optuna
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# 1. 데이터 로드
X, y = load_wine(return_X_y=True)

# 2. Objective 함수 정의
def objective_rf(trial):
    # 튜닝할 하이퍼파라미터들의 탐색 공간 정의
    n_estimators = trial.suggest_int('n_estimators', 50, 400)
    max_depth = trial.suggest_int('max_depth', 2, 32, log=True) # 로그 스케일로 탐색
    min_samples_split = trial.suggest_float('min_samples_split', 0.1, 1.0)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    # 제안받은 하이퍼파라미터로 모델 생성
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        criterion=criterion,
        random_state=42
    )

    # 3-Fold 교차 검증으로 정확도 계산
    score = cross_val_score(model, X, y, n_jobs=-1, cv=3)
    accuracy = score.mean()

    return accuracy

# 3. Study 생성 및 최적화 실행
# 정확도를 '최대화'하는 것이 목표
study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=100) # 100번 시도

# 결과 출력
print("최고 정확도:", study_rf.best_value)
print("최적 하이퍼파라미터:", study_rf.best_params)

[I 2025-06-20 08:16:31,798] A new study created in memory with name: no-name-58f82b81-fa27-4c05-9a6c-17a62dc8e276
[I 2025-06-20 08:16:33,691] Trial 0 finished with value: 0.9437853107344633 and parameters: {'n_estimators': 376, 'max_depth': 20, 'min_samples_split': 0.17933866076924498, 'criterion': 'gini'}. Best is trial 0 with value: 0.9437853107344633.
[I 2025-06-20 08:16:34,171] Trial 1 finished with value: 0.39887005649717516 and parameters: {'n_estimators': 111, 'max_depth': 8, 'min_samples_split': 0.9779844035521922, 'criterion': 'entropy'}. Best is trial 0 with value: 0.9437853107344633.
[I 2025-06-20 08:16:34,296] Trial 2 finished with value: 0.9382297551789077 and parameters: {'n_estimators': 266, 'max_depth': 19, 'min_samples_split': 0.21122496867696428, 'criterion': 'gini'}. Best is trial 0 with value: 0.9437853107344633.
[I 2025-06-20 08:16:34,418] Trial 3 finished with value: 0.9383239171374765 and parameters: {'n_estimators': 260, 'max_depth': 16, 'min_samples_split': 0.3

최고 정확도: 0.9550847457627119
최적 하이퍼파라미터: {'n_estimators': 81, 'max_depth': 11, 'min_samples_split': 0.24228757283052352, 'criterion': 'gini'}


위 코드는 랜덤 포레스트 모델의 정확도를 최대화하는 파라미터 조합을 100번의 시도 끝에 찾아냅니다.  

`direction='maximize'`로 설정하여 `objective_rf` 함수가 반환하는 정확도(`accuracy`)를 가장 높이는 방향으로 탐색이 진행됩니다.


#### ✏️ 연습문제 2

이번에는 캘리포니아 주택 가격 데이터셋 `fetch_california_housing`의 주택가격 예측 문제를 해결하는 `LightGBM` 회귀 모델(`LGBMRegressor`)의 성능을 최적화해 봅시다.

`LGBMRegressor`의 `n_estimators`, `learning_rate`, `num_leaves` 세 가지 하이퍼파라미터를 튜닝하세요. 

성능 지표는 `하이퍼파라미터(Hyperparameter)`를 사용하고, 이 값을 `Optuna(옵튜나)`하는 방향으로 최적화를 수행하세요. (RMSE를 최소화하는 것과 동일합니다.)

In [ ]:
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# 데이터 로드
X, y = fetch_california_housing(return_X_y=True)

def objective_lgbm(trial):
    # LGBMRegressor의 하이퍼파라미터 탐색 공간을 정의하세요.
    # n_estimators: 100 ~ 1000
    n_estimators = ?
    # learning_rate: 0.01 ~ 0.3 (로그 스케일)
    learning_rate = ?
    # num_leaves: 20 ~ 300
    num_leaves = ?

    model = ?

    # 3-Fold 교차 검증을 사용하여 'neg_root_mean_squared_error'를 계산하세요.
    score = ?
    neg_rmse = ?

    return neg_rmse

# study 객체를 생성하고 'maximize' 방향으로 50번의 trial을 실행하세요.
study_lgbm = ?

# 결과 출력 (최적 RMSE는 best_value에 -1을 곱해주면 됩니다.)
print("Best Negative RMSE:", study_lgbm.best_value)
print("Best RMSE:", -study_lgbm.best_value)
print("Best Hyperparameters:", study_lgbm.best_params)

---

### 4. 최적화 과정 엿보기: 시각화

Optuna의 가장 강력한 기능 중 하나는 최적화 과정을 시각적으로 분석할 수 있는 다양한 도구를 제공한다는 점입니다. 

이를 통해 우리는 단순히 최종 결과만 얻는 것이 아니라, 어떤 하이퍼파라미터가 중요한지, 탐색이 어떻게 진행되었는지 깊이 있게 이해할 수 있습니다.

#### 💻 코드 예시: Optuna 시각화 기능 사용하기

앞서 실행한 랜덤 포레스트 튜닝(`study_rf`) 결과를 바탕으로 몇 가지 유용한 시각화 차트를 그려보겠습니다.

In [3]:
# 시각화 라이브러리 임포트
from optuna.visualization import plot_optimization_history, plot_param_importances
from optuna.visualization import plot_parallel_coordinate, plot_contour

# 1. 최적화 과정 시각화 (Optimization History)
# 각 trial마다 성능이 어떻게 변해왔는지 보여줍니다.
plot_optimization_history(study_rf)

`하이퍼파라미터(Hyperparameter)`는 각 시도(trial)를 거치면서 최고 점수가 어떻게 개선되었는지를 보여줍니다.  초반에 성능이 급격히 오르다가 특정 시점부터 수렴하는 양상을 관찰할 수 있습니다. 

In [4]:
# 2. 하이퍼파라미터 중요도 시각화 (Parameter Importances)
# 어떤 파라미터가 성능에 가장 큰 영향을 미쳤는지 보여줍니다.
plot_param_importances(study_rf)

`하이퍼파라미터(Hyperparameter)`는 어떤 파라미터가 목표 함수 값에 가장 큰 영향을 미쳤는지 순위로 보여줍니다.  이를 통해 우리는 어떤 파라미터를 더 집중적으로 튜닝해야 할지 힌트를 얻을 수 있습니다. 

In [5]:
# 3. 파라미터 관계 시각화 (Parallel Coordinate Plot)
# 파라미터 조합과 성능 간의 관계를 한눈에 볼 수 있습니다.
plot_parallel_coordinate(study_rf)

`하이퍼파라미터(Hyperparameter)`는 어떤 파라미터 값의 범위가 높은 점수(파란색 선)로 이어지는지 경향성을 파악하는 데 유용합니다.

#### ✏️ 연습문제 3

연습문제 2에서 생성한 `study_lgbm` 객체를 사용하여, `learning_rate`와 `num_leaves` 두 파라미터 간의 관계를 보여주는 `Optuna(옵튜나)`를 그려보세요.

In [ ]:
# plot_contour 함수를 사용하세요.
# params 인자에 보고 싶은 파라미터 이름 리스트를 전달합니다.
from optuna.visualization import plot_contour

plot_contour(study_lgbm, params=['learning_rate', 'num_leaves'])